# Process extreme weather dataset

This notebook is used to derive a dataset of extreme events using the bias-corrected CORDEX data provided to Two Bears Environmental Consulting (via KR at SNAP) by Stantec, Inc.

This dataset will consist of four "extreme" variables (to start), derived at the annual scale for all available models and scenarios on the same grid as the original bias-corrected data.

The dataset will be a datacube with the following dimensions:

* variable
* model
* scenario
* year
* Y
* X

## Variables

The following variables are to be derived, at the annual scale:

* `hd`:  “Hot day” threshold -- the highest observed daily $T_{max}$ such that there are 5 other observations equal to or greater than this value.
* `cd`: “Cold day” threshold -- the lowest observed daily $T_{max}$ such that there are 5 other observations equal to or less than this value.
* `rx1day`: Max 1-day precipitation –- maximum 1-day precipitation
* `hsd`: Heavy Snow Days –- number of days with snowfall > 10 cm

## Models

The CORDEX data are created by combining a regional climate model with a global circulation model, and there are a couple different types of each represented in this dataset. The combinations are nowhere near exhaustive though, so for our purposes, it should be sufficient to just treat each unique combination as its own "model", of which there are 11:

* CCCma-CanESM2 x CCCma-CanRCM4
* CCCma-CanESM2 x SMHI-RCA4
* CCCma-CanESM2 x UQAM-CRCM5
* ICHEC-EC-EARTH x DMI-HIRHAM5
* ICHEC-EC-EARTH x SMHI-RCA4
* ICHEC-EC-EARTH x SMHI-RCA4-SN
* MPI-M-MPI-ESM-LR x MGO-RRCM
* MPI-M-MPI-ESM-LR x SMHI-RCA4
* MPI-M-MPI-ESM-LR x SMHI-RCA4-SN
* MPI-M-MPI-ESM-MR x UQAM-CRCM5
* NCC-NorESM1-M x SMHI-RCA4

## Processing

Here we now derive this dataset. The strategy will be to iterate over the datasets and read / summarize into summary `xarray.DataArray` objects with matching dimensions, and then combined into one `xarray.DataSet` to then be saved as a netCDF. 

Note - optimizations were attempted with `multiprocessing.Pool` but things seemed to be locking up.

Run the cell below to import the config file which sets paths to directories, makes common imports, etc.

In [1]:
from config import *
import indices

Create a list of arguments for filenames and requested summaries:

In [2]:
# this chunk is an artifact of multiprocessing-based optimization attempts
#  but it still serves nicely for utilizing a tqdm progress bar in serial processing
args = []

for scenario in scenarios:
    for varname in varnames:
        for model in models:
            fp = cordex_dir.joinpath(scenario, varname, temp_fn.format(scenario, varname, model))
            
            # aggregate variable names for this particular file
            aggr_varnames = aggr_varname_lu[varname]
            
            if fp.exists():
                args.append((fp, aggr_varnames, varname, scenario, model))

a function for summarizing a `DataArray` of model output for a particular variable to create a new data array of an aggregate variable:

In [3]:
def summarize(da, aggr, model, scenario):
    """Summarize a DataArray according to a specified
    aggregation function
    
    Args:
        da (xarray.DataArray): the DataArray object containing the base variable data to b summarized according to aggr
        aggr (str): String corresponding to the type of aggregation to do, based on the new aggregation functions
        scenario (str): scenario being run (for new coordinate dimension)
        model (str): model being run (for new coordinate dimension)
            
    Returns:
        A new data array with dimensions year, latitude, longitude, in that order containing the summarized information
    """
    new_da = (
        indices.aggr_func_lu[aggr](da)
        .transpose("time", "lat", "lon")
        .reset_coords(["longitude", "latitude", "height"], drop=True)
    )
    new_da.name = aggr
    
    # add model and scenario coordinate dimensions to the data array
    coords_di = {
        "model": model,
        "scenario": scenario,
    }

    new_dims = list(coords_di.keys())
    new_da = new_da.assign_coords(coords_di).expand_dims(new_dims)
    
    return new_da

And a wrapper function that will read in the data and run as many extreme variable summaries as requested for that particular model variable:

In [4]:
def run_summary(args):
    """Read in data and run all requested aggregations for a particular model variable, scenario, and model.
    
    Args:
        fp (path-like): path to the file for the variable required for creating the aggregate variables in aggr_varnames
        aggr_varnames (list): aggregate variables to derive using data in provided filepath
        varname (str): model variable being used for aggr_varnames
        scenario (str): scenario being run
        model (str): model being run
        
    Returns:
        summary_das (tuple): tuple of the form (da, aggr_varname, scenario, model), where da is a DataArray with dimensions of year (summary year), latitude (lat) and longitude (lon)
    """
    fp, aggr_varnames, varname, scenario, model = args
    # passing in model, scenario, agregate variable name
    #  so this information can be handed back after
    #  pool-ing to then construct new Dataset
    
    with xr.open_dataset(fp) as ds:
        out = [summarize(ds[varname], aggr, model, scenario) for aggr in aggr_varnames]
    
    return out

Iterate over the arguments created for each summary and run:

In [ ]:
import tqdm


results = []
# for arg in tqdm.tqdm(args):
#     results.extend(run_summary(*arg))
    

from multiprocessing import Pool

with Pool(8) as pool:
    for summary_da in tqdm.tqdm(
        pool.imap_unordered(run_summary, args), total=len(args)
    ):
        results.append(summary_da)

 62%|███████████████████████████████████████████████████████████████▏                                      | 65/105 [1:16:43<57:44, 86.62s/it]

Merge the `DataArray`s into one `DataSet`:

In [24]:
ds = xr.merge([da for da in results])

Set reasonable data types, converting `nan`s to -9999 as necessary:

In [55]:
def replace_nan(da):
    da.values[np.isnan(da.values)] = -9999
    return da


ds["hsd"] = replace_nan(ds["hsd"]).astype(np.int32)

Round to reasonable precision:

In [33]:
ds["hd"] = np.round(ds["hd"], 1)
ds["cd"] = np.round(ds["cd"], 1)
ds["rx1day"] = np.round(ds["rx1say"], 1)

Add metadata as attributes of the `DataArray`s

In [92]:
ds["hd"].attrs = {
    "units": "C",
    "comment": "'hot day': 6th hottest day of the year",
}
ds["cd"].attrs = {
    "units": "C",
    "comment": "'cold day': 6th coldest day of the year",
}
ds["rx1day"].attrs = {
    "units": "mm",
    "comment": "maximum precipitation for the year",
}
ds["hsd"].attrs = {
    "units": "day",
    "comment": "number of days exceeding 10cm snowfall",
    "_FillValue": -9999,
}

Global metadata:

In [93]:
from datetime import datetime


ds.attrs = {
    "creation_date": datetime.utcnow().strftime('%Y-%m-%dT%H:%M:%SZ')
}

Write to disk:

In [98]:
ds.to_netcdf(out_ds_fp)